In [2]:
from typing import List

## Bucket Sort

### Overview

**Principle idea**: 
1. Set up an array of initially empty "buckets".
2. **Scatter**: Go over the original array, putting each object in its bucket.
3. Sort each non-empty bucket.
4. **Gather**: Visit the buckets in order and put all elements back into the original array.

**Pseudo code**:
```c
function bucketSort(array, k) is
    buckets ← new array of k empty lists
    M ← 1 + the maximum key value in the array
    for i = 0 to length(array) do
        insert array[i] into buckets[floor(k × array[i] / M)]
    for i = 0 to k do 
        nextSort(buckets[i])
    return the concatenation of buckets[0], ...., buckets[k]
```

<font color=orange>The key is how to determine how many buckets to create and how to determine the range of buckets, and then how to put the elements into the corresponding bucket.</font> 

**Implementation steps:**

There are many different ways, and we introduce one method as an example. <br/>
(for more methods, see [part 2. Applications of Bucket Sort](##Applications-of-Bucket-Sort))

- **The number of buckets is equal to the number of the elements**. 
- **Except that the last bucket only contains the maximum element** in the original array, the interval range of buckets is determined by the proportion:<br/>
   `interval span (size) = (max - min) / (number of buckets - 1)`
- Find the corresponding bucket, except for the maximum elements.<br/>
  `bucket index = floor((number of buckets - 1) * [(array[i] - min) / (max - min)]) `<br/>
               `= floor((array[i] - min) / interval span)`

   ![bucket sort](https://mmbiz.qpic.cn/mmbiz_png/NtO5sialJZGoiafQsC1MKJHKiaePwNGlaCkeicz5VeTtIU1f94IHoavoJiaUcJzibEzHlENiaRvGNb3Nm9XWj7Vs6ab5g/640?wx_fmt=png&wxfrom=5&wx_lazy=1&wx_co=1)
   

### Implementation

In [32]:
def insertion_sort(arr: List[int]) -> List[int]:
    for i in range(len(arr)):
        tmp = arr[i]
        j = i - 1
        while j >= 0 and arr[j] > tmp:
            arr[j + 1] = arr[j]
            j -= 1
        arr[j + 1] = tmp
    return arr

def shell_sort(arr: List[int]) -> List[int]:
    step = 0
    while step <= (len(arr) // 3): step = step * 3 + 1
    while step >= 1:
        for i in range(step, len(arr)):
            j = i
            while j > 0 and arr[j- step] > arr[j]:
                arr[j - step], arr[j] = arr[j], arr[j - step]
                j -= step
        step //= 3
    return arr

def bucket_sort(arr: List[int]) -> List[int]:
    # Create buckets
    num_b = len(arr)
    buckets = [[] for _ in range(num_b)]
    interval = (max(arr) - min(arr)) / (num_b - 1)
    # put elements into buckets in turn
    for i in range(len(arr)):
        if arr[i] == max(arr):
            buckets[-1].append(arr[i])
            continue
        idx_b = int((arr[i] - min(arr)) / interval)  # rounding down
        buckets[idx_b].append(arr[i])
    # Sort non-empty buckets
    for i in range(num_b):
        buckets[i] = shell_sort(buckets[i])
    # Gather all elements together
    return [item for bucket in buckets for item in bucket]

In [33]:
arr = [54, 26, 93, 17, 77, 31, 44, 55, 20]
sorted_arr = bucket_sort(arr)
sorted_arr

[17, 20, 26, 31, 44, 54, 55, 77, 93]

### Analysis

#### Complexity Analysis

The computational complexity depends on the algorithm used to sort each bucket, the number of buckets to use, and whether the input is uniformly distributed.  

- **Time complexity**:
  - Worst case: all elements are placed in a single bucket. The overall performance will be dominated by the sorting algorithm, $O(N^2)$ for insertion sort and $O(N \log N0$ for comparison sort.
  - Average case:<br/>
    Consider the case that the input is uniformly distributed. 
    1. The first step, which is initialize the buckets and find the maximum key value in the array, can be done in $O(N)$ time. 
    2. If division and multiplication can be done in constant time, then scattering each element to its bucket also costs O(N). 
    3. Assume insertion sort is used to sort each bucket, then the third step costs $O ( \sum_{i}^k = {n_i}^2 )$ , where $n_{i}$ is the length of the bucket indexed $i$. Since we are concerning the average time, the expectation ${\displaystyle E(n_{i}^{2})}$ has to be evaluated instead. Let $X_{ij}$ be the random variable that is $1$ if element $j$ is placed in bucket $i$, and $0$ otherwise. We have ${\displaystyle n_{i}=\sum _{j=1}^{n}X_{ij}}$. Therefore, 
    $$
    {\displaystyle {\begin{aligned}E(n_{i}^{2})&=E\left(\sum _{j=1}^{n}X_{ij}\sum _{k=1}^{n}X_{ik}\right)\\&=E\left(\sum _{j=1}^{n}\sum _{k=1}^{n}X_{ij}X_{ik}\right)\\&=E\left(\sum _{j=1}^{n}X_{ij}^{2}\right)+E\left(\sum _{1\leq j,k\leq n}\sum _{j\neq k}X_{ij}X_{ik}\right)\end{aligned}}}
    $$
    $$
    {\displaystyle E(X_{ij}^{2})=1^{2}\cdot \left({\frac {1}{k}}\right)+0^{2}\cdot \left(1-{\frac {1}{k}}\right)={\frac {1}{k}}}
    \\
    {\displaystyle E(X_{ij}X_{ik})=1\cdot \left({\frac {1}{k}}\right)\left({\frac {1}{k}}\right)={\frac {1}{k^{2}}}}
    $$
    $$
    {\displaystyle E\left(\sum _{j=1}^{n}X_{ij}^{2}\right)+E\left(\sum _{1\leq j,k\leq n}\sum _{j\neq k}X_{ij}X_{ik}\right)=n\cdot {\frac {1}{k}}+n(n-1)\cdot {\frac {1}{k^{2}}}={\frac {n^{2}+nk-n}{k^{2}}}}
    $$
    $$
    {\displaystyle O\left(\sum _{i=1}^{k}E(n_{i}^{2})\right)=O\left(\sum _{i=1}^{k}{\frac {n^{2}+nk-n}{k^{2}}}\right)=O\left({\frac {n^{2}}{k}}+n\right)}
    $$

    The last step of bucket sort, which is concatenating all the sorted objects in each buckets, requires $O(k)$ time. Therefore, the total complexity is ${\displaystyle O\left(n+{\frac {n^{2}}{k}}+k\right)}$.
    
    Note that if k is chosen to be ${\displaystyle k=\Theta (n)}$, then bucket sort runs in $O(n)$ average time, given a uniformly distributed input
- **Space complexity**:<br/>
  Worst case: The Auxiliary Space of bucket sort is $O(n + k)$. This is because we need to create a new array of size $k$ to store the buckets and another array of size $n$ to store the sorted elements.

#### Characteristics


**distribution sort & comparison sort**: bucket sort can be implemented with comparisons and therefore can also be considered a comparison sort algorithm. 


#### Optimization


 common optimization is to put the unsorted elements of the buckets back in the original array first, then run <u>insertion sort over the complete array</u>; **because insertion sort's runtime is based on how far each element is from its final position, the number of comparisons remains relatively small,** and the memory hierarchy is better exploited by storing the list contiguously in memory.

If the input distribution is known or can be estimated, buckets can often be chosen which contain **constant density** (rather than merely having constant size). This allows $O(n)$ average time complexity **even without uniformly distributed input**. 

---

## Variants of Bucket Sort

### Predefine the number of buckets

**Implementation steps:**

- **Predefine the number of buckets $k$, randomly or according to the input**. 
- Set the interval range of buckets is determined by the proportion:<br/>
   `interval span (size) = ((max + 1) - min)) / k`<br/>
   If $\max - \min$ is used instead of $\max + 1$, the range of the last bucket is $[\max - \text{range}, \max)$, *i.e.*, the maximum element cannot be taken into the last bucket during scattering. 

- Find the corresponding bucket, except for the maximum elements.<br/>
  `bucket index = floor(k * [(array[i] - min) / (max + 1 - min)]) `<br/>
               `= floor((array[i] - min) / interval span)`

   ![bucket sort](https://upload.wikimedia.org/wikipedia/commons/thumb/6/61/Bucket_sort_1.svg/311px-Bucket_sort_1.svg.png) becomes ![](https://upload.wikimedia.org/wikipedia/commons/thumb/e/e3/Bucket_sort_2.svg/311px-Bucket_sort_2.svg.png)

In [47]:
import random
from math import ceil

def bucket_sort_general(arr: List[int]) -> List[int]:
    # Create buckets
    num_b = random.randint(3, len(arr))
    buckets = [[] for _ in range(num_b)]
    interval = ceil((max(arr) + 1 - min(arr)) / num_b)
    # put elements into buckets in turn
    for i in range(len(arr)):
        idx_b = int((arr[i] - min(arr)) / interval)  # rounding down
        buckets[idx_b].append(arr[i])
    # Sort non-empty buckets
    for i in range(num_b):
        buckets[i] = shell_sort(buckets[i])
    # Gather all elements together
    return [item for bucket in buckets for item in bucket]

In [48]:
arr = [54, 26, 93, 17, 77, 31, 44, 55, 20]
sorted_arr = bucket_sort_general(arr)
sorted_arr

[17, 20, 26, 31, 44, 54, 55, 77, 93]